<a href="https://colab.research.google.com/github/sharik31/SQL-Generator/blob/main/sql_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets peft bitsandbytes accelerate
!pip uninstall -y transformers accelerate
!pip install -U "transformers>=4.41.0" "accelerate>=0.33.0" "datasets>=2.20.0" evaluate peft bitsandbytes
!pip install -U transformers datasets peft accelerate trl




In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig

model_name = "t5-small"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"

model = get_peft_model(model, lora_config)


In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "csv",
    data_files={
        "train": "/content/WikiSQL.csv",
        "validation": "/content/Wikisql_val.csv",
        "test": "/content/Wikisql_test.csv"
    }
)

def preprocess(examples):
    inputs = tokenizer(examples["question"], truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(examples["sql"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True)


In [ ]:
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs",
    use_legacy_prediction_loop=False,
)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")




In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    processing_class=tokenizer,
    data_collator=data_collator
)





In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("lora_sql_model")


In [ ]:
question = "Retrieve all rolls from the student table."
tables = "ADMIN: USERNAME (PRIMARY KEY) (text); PASS (text); STUDENT: roll (PRIMARY KEY) (int), name (text), dept (text)"

input_text = "Here is schema details: " + tables + " Answer this question as an SQL query: " + question

inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=1024).to(device)
outputs = model.generate(**inputs, max_length=512)
sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Predicted SQL:", sql_query)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

base_model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name, device_map="auto")

model = PeftModel.from_pretrained(base_model, "lora_sql_model", device_map="auto")
model.eval()

In [ ]:
import pandas as pd
from tqdm import tqdm
import torch

df_test = pd.read_csv("/content/Wikisql_test.csv")

df_test = df_test.iloc[:1000]

inputs = df_test['question'].tolist()
labels = df_test['sql'].tolist()


device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


batch_size = 16
generated_sql_queries = []

for i in tqdm(range(0, len(inputs), batch_size)):
    batch_texts = inputs[i:i+batch_size]

    input_tensor = tokenizer(batch_texts,
                             return_tensors="pt",
                             truncation=True,
                             padding=True,
                             max_length=1024).to(device)

    outputs = model.generate(**input_tensor, max_length=512)


    batch_sql = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    generated_sql_queries.extend(batch_sql)

for i in range(10):
    print(f"Question: {inputs[i]}")
    print(f"Predicted SQL: {generated_sql_queries[i]}")
    print(f"Ground Truth SQL: {labels[i]}")
    print("-"*40)



In [ ]:
import nltk
import evaluate

nltk.download("punkt", quiet=True)
nltk.download("punkt_tab", quiet=True)
rouge = evaluate.load("rouge")
decoded_preds = ["\n".join(nltk.sent_tokenize(p.strip())) for p in generated_sql_queries]
decoded_labels = ["\n".join(nltk.sent_tokenize(l.strip())) for l in labels]
rouge_score = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

print("\nROUGE Score on Test Set:")
print(rouge_score)

In [ ]:
!pip install rouge_score